# Sample Backtest for Futures SMA Strategy on binance 

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from itertools import product
import warnings
warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8")

In [33]:
class FuturesSMABacketest():

    def __init__(self,  symbol, start_date, trading_costs):
        self.symbol = symbol
        self.start_date = start_date
        self.trading_costs = trading_costs
        self.results = None
        self.data_df = self.get_data()
        self.tp_year = (self.data_df.close.count() / ((self.data_df.index[-1] - self.data_df.index[0]).days / 365.25))
        

    def __repr__(self):
        return "Futures_Backtester(symbol = {}, start = {})".format(self.symbol, self.start_date)
    
    def get_data(self):
        url = f"https://91j3dag4m9.execute-api.us-east-1.amazonaws.com/master/getCMCPriceHistory/1?startDate={self.start_date}"
        response = requests.get(url)
        data = response.json()
        table = []
        for entry in data["priceHistory"]:
            table.append([
                entry['d'],
                entry["c"],
                entry["v"],
            ])

        columns = ["date", "close", "volume" ]
        df = pd.DataFrame(table, columns=columns)
         # Convert 'date' column to datetime format
        df['date'] = pd.to_datetime(df['date'])
        # Set 'date' as the index
        df.set_index('date', inplace=True)
        df['returns'] = np.log(df.close / df.close.shift(1))
        return df
    
    def prepare_data(self, smas):


        # Here we'll generate the positions
        data = self.data_df[['close', 'returns']].copy()
        data["sma_short"] = data.close.rolling(window = 15).mean()
        data["sma_medium"] = data.close.rolling(window = 50).mean()
        data["sma_long"] = data.close.rolling(window = 200).mean()

        data.dropna(inplace = True)
    
        # Condition 1 means short sma crossed medium to the upside 
        cond1 = (data.sma_short > data.sma_medium) & (data.sma_medium > data.sma_long)
        cond2 = (data.sma_short < data.sma_medium) & (data.sma_medium < data.sma_long)
        
        data["position"] = 0
        data.loc[cond1, "position"] = 1
        data.loc[cond2, "position"] = -1

        self.results = data

    def run_backtest(self):

        data = self.results.copy()
        data['strategy'] = data.position.diff().fillna(0).abs()
        data['trades'] = data.position.diff().fillna(0)
        data["strategy"] = data["position"].shift(1) * data["returns"]
        data["trades"] = data.position.diff().fillna(0).abs()
        data.strategy = data.strategy + data.trades * self.trading_costs

        self.results = data

    def test_strategy(self, smas):
        '''
        Prepares the data and backtests the trading strategy incl. reporting (Wrapper).
         
        Parameters
        ============
        smas: tuple (SMA_S, SMA_M, SMA_L)
            Simple Moving Averages to be considered for the strategy.
            
        '''
        
        self.sma_short = smas[0]
        self.sma_medium = smas[1]
        self.sma_long = smas[2]
        
        
        self.prepare_data(smas = smas)
        self.run_backtest()
        
        data = self.results.copy()
        data["creturns"] = data["returns"].cumsum().apply(np.exp)
        data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
        self.results = data
        
        self.print_performance()

    def plot_results(self, leverage = False): #Adj!
        '''  Plots the cumulative performance of the trading strategy compared to buy-and-hold.
        '''
        if self.results is None:
            print("Run test_strategy() first.")
        elif leverage: 
            title = "{} | Trading Costs = {} | Leverage = {}".format(self.symbol, self.trading_costs, self.leverage)
            self.results[["creturns", "cstrategy", "cstrategy_levered"]].plot(title=title, figsize=(12, 8))
        else:
            title = "{} | Trading Costs = {}".format(self.symbol, self.trading_costs)
            self.results[["creturns", "cstrategy"]].plot(title=title, figsize=(12, 8))

    def print_performance(self, leverage = False): # Adj
        ''' Calculates and prints various Performance Metrics.
        '''
        
        data = self.results.copy()
        
        if leverage: # NEW!
            to_analyze = np.log(data.strategy_levered.add(1))
        else: 
            to_analyze = data.strategy
            
            
        strategy_multiple = round(self.calculate_multiple(to_analyze), 6)
        bh_multiple =       round(self.calculate_multiple(data.returns), 6)
        outperf =           round(strategy_multiple - bh_multiple, 6)
        cagr =              round(self.calculate_cagr(to_analyze), 6)
        ann_mean =          round(self.calculate_annualized_mean(to_analyze), 6)
        ann_std =           round(self.calculate_annualized_std(to_analyze), 6)
        sharpe =            round(self.calculate_sharpe(to_analyze), 6)
       
        print(100 * "=")
        print("TRIPLE SMA STRATEGY | INSTRUMENT = {} | SMAs = {}".format(self.symbol, [self.sma_short, self.sma_medium, self.sma_long]))
        print(100 * "-")
        print("PERFORMANCE MEASURES:")
        print("\n")
        print("Multiple (Strategy):         {}".format(strategy_multiple))
        print("Multiple (Buy-and-Hold):     {}".format(bh_multiple))
        print(38 * "-")
        print("Out-/Underperformance:       {}".format(outperf))
        print("\n")
        print("CAGR:                        {}".format(cagr))
        print("Annualized Mean:             {}".format(ann_mean))
        print("Annualized Std:              {}".format(ann_std))
        print("Sharpe Ratio:                {}".format(sharpe))
        
        print(100 * "=")

    def calculate_multiple(self, series):
        return np.exp(series.sum())
    
    def calculate_cagr(self, series):
        return np.exp(series.sum())**(1/((series.index[-1] - series.index[0]).days / 365.25)) - 1
    
    def calculate_annualized_mean(self, series):
        return series.mean() * self.tp_year
    
    def calculate_annualized_std(self, series):
        return series.std() * np.sqrt(self.tp_year)
    
    def calculate_sharpe(self, series):
        if series.std() == 0:
            return np.nan
        else:
            return self.calculate_cagr(series) / self.calculate_annualized_std(series)



In [34]:
symbol = "BTCUSDT"
start_date = '2018-01-01'
tc = -0.0005
sma_s = 15 
sma_m = 50 
sma_l = 200 

tester = FuturesSMABacketest( symbol = 'BTC',
                              start_date = start_date,  trading_costs = tc)

tester.data_df

,close,volume,returns
date,,,
2018-01-01,13657.200195,1.029120e+10,NaN
2018-01-02,14982.099609,1.684660e+10,0.092589
2018-01-03,15201.000000,1.687190e+10,0.014505
2018-01-04,15599.200195,2.178320e+10,0.025858
2018-01-05,17429.500000,2.384090e+10,0.110945
...,...,...,...
2023-09-06,25753.236258,1.275271e+10,-0.001038
2023-09-07,26240.194892,1.108831e+10,0.018732
2023-09-08,25905.653752,1.081736e+10,-0.012831


In [35]:
tester.test_strategy(smas = (sma_s, sma_m, sma_l))

AttributeError: 'FuturesSMABacketest' object has no attribute 'SMA_S'